In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
import cv2
import math
import matplotlib.pyplot as plt

In [2]:
anno_pathes_json = []
file_name = '.json'

#For home
absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\TV_data'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\basketball_numbers'

#For laptop
#absolute_path = 'D:\\Datasets\\TV_data'
#crop_path = 'D:\\Datasets\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

In [24]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []
xxx = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)
    xxx.append(anno)
    #Преобразуем json  в dataFrame
    points = pd.DataFrame(data['shapes'])
    #file_name = pd.read_json(json.dumps(data["imagePath"]))
    x = pd.read_json(anno, orient='index')   
    frame = x.iloc[3, :][0]
    frame = frame.replace('.', '-')
 
    img_path = anno.replace('anno', 'frames')
    img_path = img_path.replace('.json', '.jpeg')
    img_path = img_path.replace("BallerTV", "TV_data")
    img = Image.open(img_path)

    for j in range(len(points)):
        out_file = {}
        point_obj = pd.DataFrame(points.iloc[j, 0:2])
        point = point_obj.iloc[1, :]
        label = point_obj.iloc[0, :]
        split_label = label[j].split('_')
        number = split_label[-1]

        for box in point:   

            if number.isdigit() & (len(split_label) == 3):

                xtl, ytl, xbr, ybr = int(box[0][0]), int(box[0][1]), int(box[1][0]), int(box[1][1])
                            
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)
                

                xtl = int(xtl - math.ceil(alpha * (x_dif / 2)))
                xbr = int(xbr + math.ceil(alpha * (x_dif / 2)))
                ytl = int(ytl - math.ceil(alpha * (y_dif / 2)))
                ybr = int(ybr + math.ceil(alpha * (y_dif / 2)))

                # Сделаем кроп номера на футболке
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)                

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'TV_data_{str(frame)}_{num}example.jpg'
                num += 1
                
                croppped_image.save(res_path)

            


  0%|          | 0/5613 [00:00<?, ?it/s]

KeyError: 'shapes'

In [25]:
print(len(xxx))
print(xxx[-2])
print(xxx[-1])

5599
C:\Users\Mytre\OneDrive\Документы\Data\Work\data\TV_data\basketball\0_processed\7-05-2022\anno\Raw_07LGs4GLBKU7siUCLNcVEt3bahbhd0_1602624046.35_FishEye\013115.json
C:\Users\Mytre\OneDrive\Документы\Data\Work\data\TV_data\volleyball\00-00-00\anno\7637_92817_1656636370_raw.json


In [23]:
an = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\TV_data\\basketball\\0_processed\\7-05-2022\\anno\\Raw_07LGs4GLBKU7siUCLNcVEt3bahbhd0_1602624046.35_FishEye\\013115.json'

with open(an, 'r') as f:
    data = json.load(f)

#Преобразуем json  в dataFrame
points = data['shapes']
pd.DataFrame(points).head()

,label,points,group_id,shape_type,flags
0,1_t0,"[[577.9285714285713, 558.2857142857142], [611....",None,rectangle,{}
1,2_t0,"[[331.49999999999994, 560.0714285714286], [381...",None,rectangle,{}
2,3_t0,"[[443.99999999999994, 547.5714285714286], [481...",None,rectangle,{}
3,4_t0,"[[365.4285714285714, 561.8571428571428], [397....",None,rectangle,{}
4,6_t1,"[[525.438596491228, 546.4912280701753], [554.3...",None,rectangle,{}


In [15]:
with open(an, 'r') as f:
    class_names = json.load(f)

pd.DataFrame(class_names['shapes'])

,label,points,group_id,shape_type,flags
0,1_t0,"[[577.9285714285713, 558.2857142857142], [611....",None,rectangle,{}
1,2_t0,"[[331.49999999999994, 560.0714285714286], [381...",None,rectangle,{}
2,3_t0,"[[443.99999999999994, 547.5714285714286], [481...",None,rectangle,{}
3,4_t0,"[[365.4285714285714, 561.8571428571428], [397....",None,rectangle,{}
4,6_t1,"[[525.438596491228, 546.4912280701753], [554.3...",None,rectangle,{}
5,7_t1,"[[442.1052631578947, 583.3333333333333], [489....",None,rectangle,{}
6,j_7_22,"[[455.26315789473676, 613.1578947368421], [474...",None,rectangle,{}
7,8_t1,"[[407.0175438596491, 565.7894736842105], [435....",None,rectangle,{}
8,9_t1,"[[260.52631578947364, 585.0877192982456], [300...",None,rectangle,{}
9,10_t1,"[[649.1228070175438, 562.2807017543859], [680....",None,rectangle,{}
